<a href="https://colab.research.google.com/github/raj-coding1/comment-analysis/blob/main/Exp_6_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow dvc dagshub optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.8/467.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.

In [ ]:
import pandas as pd
# from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import mlflow
import optuna
import dvc
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = pd.read_csv('/content/df.csv')
df.head()
df.dropna(axis = 0,inplace = True)
ngram_choice= (1,3)
max_feat = 1000
vectorizer = TfidfVectorizer(
            max_features=max_feat,
            ngram_range=ngram_choice
        )
X = vectorizer.fit_transform(df['cleaned_comment'])
df["category"] = df["category"].replace(-1, 2)
y = df['category']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow')
dagshub.init(repo_owner='raj-coding1', repo_name='youtube-comment-analysis', mlflow=True)

mlflow.set_experiment('EXP-11: Final LightGBM')


def log_ml_model(model_name, model, X_train,X_test,y_train,y_test, params, trial_number):
  with mlflow.start_run():
    mlflow.set_tag('mlflow.runname',f'{model_name}_tfidf_smote_trigram')
    mlflow.set_tag("experiment_type", "algorithm comparison")
    mlflow.log_param("algo_name", model_name)

    for key,value in params.items():
      mlflow.log_param(key,value)

    # Train model

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Correct metric logging
    mlflow.log_metric("accuracy", accuracy)

    # Correct classification report
    class_rep = classification_report(y_test, y_pred, output_dict=True)

    for label, info in class_rep.items():
        if isinstance(info, dict):
            for metric, value in info.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Save confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Prediction")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")

    plt.savefig("ConfusionMatrix.png")
    mlflow.log_artifact("/content/ConfusionMatrix.png")
    plt.close()

    # Save dataset (if needed)
    df.to_csv("df1.csv", index=False)
    mlflow.log_artifact("/content/df1.csv")
    # mlflow.sklearn.log_model(model,f'{model_name}_model')
    print(f"accuracy : {accuracy}")


def objective(trial):
  n_estimators = trial.suggest_int('n_estimators',100,1000)
  learning_rate = trial.suggest_float('learning_rate',1e-4,1e-1, log = True)
  max_depth = trial.suggest_int('max_depth',3, 15)
  num_leaves = trial.suggest_int('num_leaves',20,150)
  min_child_samples = trial.suggest_int('min_child_samples',10,100)
  cosample_bytree = trial.suggest_float('cosample_bytree',0.5,1.0)
  subsample = trial.suggest_float('subsample',0.5,1.0)
  reg_alpha = trial.suggest_float('reg_alpha',1e-4,10.0,log = True)
  reg_lamda = trial.suggest_float('reg_lamda',1e-4,10.0,log = True)

  params = {
      'n_estimators' : n_estimators,
      'learning_rate' : learning_rate,
      'max_depth' : max_depth,
      'num_leaves' : num_leaves,
      'min_child_samples': min_child_samples,
      'cosample_bytree' : cosample_bytree,
      'subsample' : subsample,
      'reg_alpha' : reg_alpha,
      'reg_lamda' : reg_lamda
        }

  model = LGBMClassifier(n_estimators = n_estimators, learning_rate = learning_rate, max_depth = max_depth, random_state = 42,
                         num_leaves = num_leaves,
                         min_child_samples = min_child_samples,
                         cosample_bytree = cosample_bytree,
                         subsample = subsample,
                         reg_alpha = reg_alpha,
                         reg_lamda = reg_lamda
                         )

  accuracy = log_ml_model('LGBMClassifier', model, X_train,X_test,y_train,y_test,params, trial.number)
  return accuracy

def run_optuna_exp():
  study = optuna.create_study(direction = 'maximize')
  study.optimize(objective,n_trials = 30)
  best_params = study.best_params
  best_model = LGBMClassifier(n_estimators = best_params['n_estimators'], learning_rate = best_params['learning_rate'], max_depth = best_params['max_depth']
                              num_leaves = best_params['num_leaves'],
                              min_child_samples = best_params['min_child_samples'],
                              cosample_bytree = best_params['cosample_bytree'],
                              subsample = best_params['subsample'],
                              reg_alpha = best_params['reg_alpha'],
                              reg_lamda = best_params['reg_lamda']
                              )
  log_ml_model('LGBMClassifier', best_model, X_train,X_test,y_train,y_test, best_params, 'Best')

run_optuna_exp()

Initialized MLflow to track repo "raj-coding1/youtube-comment-analysis"

Repository raj-coding1/youtube-comment-analysis initialized!

[I 2025-11-30 11:07:55,370] A new study created in memory with name: no-name-29524a00-a119-494c-b289-18ccb3f4ecc7


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.176912 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 81446
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 972
[LightGBM] [Info] Start training from score -1.062681
[LightGBM] [Info] Start training from score -0.845356
[LightGBM] [Info] Start training from score -1.491356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-30 11:08:04,803] Trial 0 finished with value: 0.4331105959361789 and parameters: {'n_estimators': 191, 'learning_rate': 0.00025947481754215484, 'max_depth': 4}. Best is trial 0 with value: 0.4331105959361789.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.178160 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 81446
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 972
[LightGBM] [Info] Start training from score -1.062681
[LightGBM] [Info] Start training from score -0.845356
[LightGBM] [Info] Start training from score -1.491356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-30 11:15:35,689] Trial 29 finished with value: 0.7074867039410883 and parameters: {'n_estimators': 249, 'learning_rate': 0.020760587228050815, 'max_depth': 5}. Best is trial 25 with value: 0.798581753716078.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.316453 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 81446
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 972
[LightGBM] [Info] Start training from score -1.062681
[LightGBM] [Info] Start training from score -0.845356
[LightGBM] [Info] Start training from score -1.491356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


accuracy : 0.798581753716078
🏃 View run intelligent-ape-938 at: https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow/#/experiments/9/runs/c84eea504afd4e5482949242df5dc42f
🧪 View experiment at: https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow/#/experiments/9
